# Gaussian Process Factor Analysis Regression

## Introduction

Gaussian Process Factor Analysis (GPFA), introduced in [this paper](https://papers.nips.cc/paper/2008/file/ad972f10e0800b49d76fed33a21f6698-Paper.pdf) learns underlying latent trajectories for the outputs while smoothing the data. A more [recent paper](https://papers.nips.cc/paper/2020/file/6d79e030371e47e6231337805a7a2685-Paper.pdf) suggests scalable implementations for GPFA (in the [supplement](https://papers.nips.cc/paper/2020/file/6d79e030371e47e6231337805a7a2685-Supplemental.pdf) ) as well as extending GPFA with kernels for dynamical systems. The implementation in this notebook is developed with this GPFA for Dynamical Systems (GPFADS) extension in mind. 

GPFA(DS) are useful when you want to simultaneous smooth and reduce the dimensionality of neural data. 

Given timepoints $t$, $M$ latent variables $x$ with underlying kernels $k_1, ...k_M$, and observations (neural data) $y$ from $N$ neurons, the observations in GPFA are are assumed to arise as follows:
- The latents variables are independent of eachother:
$$  k[x_i(t),x_i(t')] = k_i(t,t')
$$

- We write the above in a vector of latents as:
$$k[\mathbf{x}(t), \mathbf{x}(t')] = \sum_{i=1}^{M}k_i(t,t')\otimes \mathbf{e_i}
$$

- We combine the latents into observations as follows:
$$k[\mathbf{y}_i(t), \mathbf{y}_i(t')] = (C) k( \mathbf{x}(t) , \mathbf{x}(t'))(C^T)
$$

where $k_i$ is a standard kernel (e.g. RBF) that operates on the inputs.
$\mathbf{e}_i$ is a unit vector of length $M$ with nonzero element at index $i$, $\otimes$ is the Kronecker product, and $C\in NxM$ is the mixing matrix, indicating how latents are combined into observations.

In [1]:
import math
import torch
import gpytorch
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

### Set up training data
In the next cell, we set up the training data for this example. We'll be using 1000 regularly spaced points on [0,100], our data will be drawn from the generative model assumed in GPFA.

In [2]:
def generate_GPFA_Data(seed=0, n_timepoints=100, n_latents=2, n_outputs=50, length_scales = [.01,10],start_time=-5, end_time = 5,zero_mean = True):
    torch.manual_seed(seed)
    np.random.seed(seed)
    timepoints = torch.linspace(start_time, end_time, n_timepoints)
    tau = torch.tensor(length_scales)
    C = torch.tensor(np.random.normal(scale=1./np.sqrt(n_latents),size=(n_outputs,n_latents))).float()
    if zero_mean:
        d = torch.zeros(size=(n_outputs,1)).float()
    else:
        d = torch.tensor(np.random.uniform(size=(n_outputs,1))).float()
    R = torch.tensor(np.diag(np.random.uniform(size=(n_outputs,), low=.1))).float()
    kernels = [gpytorch.kernels.RBFKernel() for t in tau]
    for t in range(len(tau)):
        kernels[t].lengthscale = tau[t]

    xs = torch.stack([
        gpytorch.distributions.MultivariateNormal(
            torch.zeros(n_timepoints), 
            k(timepoints,timepoints) 
        ).sample()
        for k in kernels])

    ys = gpytorch.distributions.MultivariateNormal((C@xs + d).T, R).sample()
    
    xs = xs.T.contiguous()
    
    return timepoints, tau, C, d, R, kernels, xs, ys

In [3]:
n_latents=2
n_outputs=20
n_timepoints = 100
length_scales = [.1,.2]
start_time = 0
end_time = 1
train_x, tau, C, d, R, kernels, xs, train_y = generate_GPFA_Data(
    seed=10, n_timepoints=n_timepoints, n_latents=n_latents, n_outputs=n_outputs, 
    length_scales = length_scales,start_time = start_time, end_time = end_time, zero_mean =True)

/scratch/jts58/devenv/lib/python3.7/site-packages/gpytorch-1.4.0-py3.7.egg/gpytorch/utils/cholesky.py:51: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(f"A not p.d., added jitter of {jitter_new:.1e} to the diagonal", NumericalWarning)


## Define a multitask model

The model should be somewhat similar to the `ExactGP` model in the [simple regression example](../01_Exact_GPs/Simple_GP_Regression.ipynb).
The differences:

1. We're going to wrap ConstantMean with a `MultitaskMean`. This makes sure we have a mean function for each output dimension.
2. Rather than just using a RBFKernel, we're using that in conjunction with a `GPFAKernel`. This gives us the covariance function described in the introduction.
3. We're using a `MultitaskMultivariateNormal` and `MultitaskGaussianLikelihood`. This allows us to deal with the predictions/outputs in a nice way. For example, when we call MultitaskMultivariateNormal.mean, we get a `T x N` matrix back (where `T` is the number of timepoints).


- [ ] TODO: verify below

You may also notice that we don't use a ScaleKernel, since the GPFAKernel will do some scaling for us. (This way we're not overparameterizing the kernel.)

In [4]:
from gpytorch.kernels import GPFAKernel
class GPFAModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, latent_covar_modules,  num_latents, num_obs):
        #TODO: infer number latents / obs from train_x / train_y?
        super(GPFAModel, self).__init__(train_x, train_y, likelihood)
        
        self.num_latents = num_latents
        self.num_obs = num_obs
        
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=num_obs
        )
        self.covar_module = GPFAKernel(latent_covar_modules,num_latents, num_obs)
        
    def forward(self, x):
        return gpytorch.distributions.MultitaskMultivariateNormal(self.mean_module(x), self.covar_module(x))


In [5]:
likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=n_outputs)
kernels = [gpytorch.kernels.RBFKernel() for t in range(n_latents)]
model=GPFAModel(train_x,train_y, likelihood,  kernels, n_latents, n_outputs)

- [ ] TODO: make num obs / outputs consistent wording

### Train the model hyperparameters

In [15]:
# this is for running the notebook in our testing framework
import os
smoke_test = ('CI' in os.environ)
training_iterations = 2 if smoke_test else 500


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
with gpytorch.settings.max_preconditioner_size(0), gpytorch.settings.verbose_linalg(True),gpytorch.settings.fast_computations(False,False,False),gpytorch.settings.cg_tolerance(1e-2):
    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f'
              #, Lengthscale 1: %.3f, Lengthscale 2: %.3f' 
              % (i + 1, training_iterations, loss.item()))#, kernels[0].lengthscale.float(), kernels[1].lengthscale.float()))
        optimizer.step()

LinAlg (Verbose) - DEBUG - Running Cholesky on a matrix of size torch.Size([2000, 2000]).
LinAlg (Verbose) - DEBUG - Running Cholesky on a matrix of size torch.Size([2000, 2000]).


Iter 1/500 - Loss: 117.796


LinAlg (Verbose) - DEBUG - Running Cholesky on a matrix of size torch.Size([2000, 2000]).


Iter 2/500 - Loss: 118.443


LinAlg (Verbose) - DEBUG - Running Cholesky on a matrix of size torch.Size([2000, 2000]).


...

Iter 497/500 - Loss: 105.787


LinAlg (Verbose) - DEBUG - Running Cholesky on a matrix of size torch.Size([2000, 2000]).


Iter 498/500 - Loss: 105.786


LinAlg (Verbose) - DEBUG - Running Cholesky on a matrix of size torch.Size([2000, 2000]).


Iter 499/500 - Loss: 105.786
Iter 500/500 - Loss: 105.785


In [16]:
print(kernels[0].lengthscale, kernels[1].lengthscale
     )

tensor([[0.4972]], grad_fn=<SoftplusBackward>) tensor([[0.1023]], grad_fn=<SoftplusBackward>)


In [20]:
likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=n_outputs)
likelihood.noise = 1e-4
likelihood.task_noises = torch.diagonal(R) - 1e-4





kernels = [gpytorch.kernels.RBFKernel() for t in range(n_latents)]
for t in range(n_latents):
    kernels[t].lengthscale = length_scales[t]
model=GPFAModel(train_x,train_y, likelihood,  kernels, n_latents, n_outputs)
model.covar_module.C = C
for i in range(n_outputs):
    model.mean_module.base_means[i].constant = torch.nn.Parameter(d[i])


AttributeError: 'NoneType' object has no attribute '_validate_sample'

In [22]:
help(gpytorch.likelihoods.MultitaskGaussianLikelihood)

Help on class MultitaskGaussianLikelihood in module gpytorch.likelihoods.multitask_gaussian_likelihood:

class MultitaskGaussianLikelihood(_MultitaskGaussianLikelihoodBase)
 |  MultitaskGaussianLikelihood(num_tasks, rank=0, task_prior=None, batch_shape=torch.Size([]), noise_prior=None, noise_constraint=None, has_global_noise=True, has_task_noise=True)
 |  
 |  A convenient extension of the :class:`gpytorch.likelihoods.GaussianLikelihood` to the multitask setting that allows
 |  for a full cross-task covariance structure for the noise. The fitted covariance matrix has rank `rank`.
 |  If a strictly diagonal task noise covariance matrix is desired, then rank=0 should be set. (This option still
 |  allows for a different `noise` parameter for each task.)
 |  
 |  Like the Gaussian likelihood, this object can be used with exact inference.
 |  
 |  Method resolution order:
 |      MultitaskGaussianLikelihood
 |      _MultitaskGaussianLikelihoodBase
 |      gpytorch.likelihoods.gaussian_like